In [1]:
# working dir
import os
import sys

cwd = os.getcwd()
root_dir = os.path.dirname(os.path.dirname(cwd))
sys.path.append(root_dir)

print(root_dir)

c:\Users\david\Desktop\Main\03_Uni\WWI21DSA\02_Vorlesungen\06_Projektrealisierung\Projektrealisierung


In [2]:
# Imports and settings
import pandas as pd
import random

random.seed(42)

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)

In [14]:
# Load data
abcd = pd.read_csv("../../src/data/ABCD_tripfiles_preprocessed.csv")
mnop = pd.read_csv("../../src/data/MNOP_tripfiles_preprocessed.csv")
zyxw = pd.read_csv("../../src/data/ZYXW_tripfiles_preprocessed.csv")

C:\Users\david\AppData\Local\Temp\ipykernel_27460\136873643.py:3: DtypeWarning: Columns (12,13,14,15,16,17,18,19,20,21,22,23,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  mnop = pd.read_csv("../../src/data/MNOP_tripfiles_preprocessed.csv")
C:\Users\david\AppData\Local\Temp\ipykernel_27460\136873643.py:4: DtypeWarning: Columns (12,13,14,15,16,17,18,19,20,21,22,24,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  zyxw = pd.read_csv("../../src/data/ZYXW_tripfiles_preprocessed.csv")


In [4]:
# Subset of data
random_flights = random.sample(abcd["header_id"].tolist(), 250)
abcd_subset = abcd[abcd["header_id"].isin(random_flights)]

In [5]:
meta = ['id', 'creation_time', 'airline_code', 'flight_number', 'flight_date',
       'departure_airport', 'user_name', 'action_name']

message = ['header_line', 'entry_details']

header_cols = ['header_category', 'header_id']

extracted_cols = ['scheduleState',
       'departureAirport', 'departureTime', 'arrivalAirport', 'arrivalTime',
       'aircraftRegistration', 'aircraftSubtype', 'aircraftVersion',
       'serviceType', 'nonOperational', 'flifoCancel', 'sequenceNumber',
       'bestTimes', 'mvtActual']

weight_cols = ['START_WI_weight', 'START_WI_index',
       'DO_WI_weight', 'DO_WI_index', 'PAX_WI_weight', 'PAX_WI_index',
       'TOTAL_DEADLOAD_WI_weight', 'TOTAL_DEADLOAD_WI_index', 'TOTAL_LOAD_WI',
       'TOTAL_TRAFFIC_LOAD', 'FUEL_INDEX', 'AZFW', 'ATOW', 'ALAW', 'ATXW',
       'AFT_LIMIT_ZFW', 'FWD_LIMIT_ZFW', 'AFT_LIMIT_TOW', 'FWD_LIMIT_TOW',
       'AFT_LIMIT_LAW', 'FWD_LIMIT_LAW', 'LIZFW', 'LITOW', 'LILAW',
       'MAC_AT_ZFW', 'MAC_AT_TOW', 'MAC_AT_LAW', 'DEADLOAD_MAC', 'UNDERLOAD',
       'LIMITING_WEIGHT', 'ALLOWED_TOW', 'ALLOWED_ZFW', 'ALLOWED_LAW',
       'ALLOWED_TXW', 'STABTO', 'OPTIMAL_TRIM', 'IDEAL_ADDITIONAL_LOAD_AFT',
       'IDEAL_ADDITIONAL_LOAD_FWD', 'TAIL_TIPPING_WI_weight',
       'TAIL_TIPPING_WI_index', 'TAIL_TIPPING_INDEX_EXCEEDED',
       'FWD_MOVABLE_PAX', 'AFT_MOVABLE_PAX', 'INDEX_OUT_OF_BALANCE',
       'LOAD_TO_AFT', 'LOAD_TO_FWD', 'ESTIMATED_TRAFFIC_LOAD', 'ESTIMATED_ZFW',
       'DELTA_ZFW', 'ZFW_TOLERANCE_EXCEEDED']


In [6]:
# Zeigt wie oft die einzelnen actions pro flug vorkommen
action_occurences = abcd_subset.drop_duplicates(subset=['header_id', 'action_name'])
action_occurences['action_name'].value_counts()

action_name
CalculateWeightAndTrimAction           234
StorePaxDataAction                     175
CheckinMsgProcessor                    147
UpdateCrewDataAction                    30
ASMMsgProcessor                         27
UpdateLoadTableAction                   25
UpdateFlightAction                      20
AssignLCCAction                         20
StoreRegistrationAndConfigurationAc     18
ResetLoadingListRecordsAction           15
CrewMsgProcessor                        12
EstimateStorePaxDataAction              11
PAXBOOKINGINMsgProcessor                 9
TransferCheckinDataAction                8
ChangeFlightLegStateAction               6
CreatePostDepartureMessagesAction        6
UpdateHistoricalFlightLegInfoAction      5
CloseLegAction                           5
InternalCreateLoadingInstructionAct      5
SendPostDepartureMessagesAction          5
InternalCreateLoadsheetAction            5
CreateLoadingInstructionAction           4
UpdateSupplementaryInfoAction            4

### >> Finding 1: Wenn man header_id als index nimmt -> Von den 250 flights im subset hatten 234 einen Eintrag für die Action CalculateWeightAndTrimAction. Die anderen Aktionen kamen seltener vor und stehen dementsprechend für weniger flüge zur Verfügung.

In [7]:
# create makeshift flight identifier
abcd["temp"] = abcd["departureAirport"] + abcd["arrivalAirport"] + abcd["departureTime"] + abcd["arrivalTime"]

In [8]:
# Wie viele Flüge gäbe es, bei den unterschiedlichen identifizierungsansätzen
print(
    len(abcd),

    abcd["id"].nunique(),
    abcd["header_id"].nunique(),
    abcd["flight_number"].nunique(),
    abcd["temp"].nunique()
)

709613 709613 121282 163 2379


In [9]:
# Stichprobenartig einen "flug" (nach temp index) anschauen
temp_table = abcd[abcd["temp"]=="BLRDEL2024-05-01T23:00:00.000Z2024-05-02T01:50:00.000Z"]

temp_table[["header_id", "header_category"] + meta + ['departureAirport', 'departureTime', 'arrivalAirport', 'arrivalTime',
       'aircraftRegistration', 'aircraftSubtype', 'aircraftVersion']].sort_values("creation_time")

,header_id,header_category,id,creation_time,airline_code,flight_number,flight_date,departure_airport,user_name,action_name,departureAirport,departureTime,arrivalAirport,arrivalTime,aircraftRegistration,aircraftSubtype,aircraftVersion
207660,9d1a19e10783012d,saved,33846020,2024-04-30 22:14:13,AB,2359,1,BLR,service-acco,CalculateWeightAndTrimAction,BLR,2024-05-01T23:00:00.000Z,DEL,2024-05-02T01:50:00.000Z,ABCAD,7MA,Y189
207661,9d1a19e10783012d,received,33845797,2024-04-30 22:14:13,AB,2359,1,BLR,service-acco,CalculateWeightAndTrimAction,BLR,2024-05-01T23:00:00.000Z,DEL,2024-05-02T01:50:00.000Z,ABCAD,7MA,Y189
207662,9d1a19e10783012d,received,33845795,2024-04-30 22:14:13,AB,2359,1,BLR,service-acco,CalculateWeightAndTrimAction,BLR,2024-05-01T23:00:00.000Z,DEL,2024-05-02T01:50:00.000Z,ABCAD,7MA,Y189
480043,9d1a19e10783012d,received,33845792,2024-04-30 22:14:13,AB,2359,1,BLR,service-acco,ResetLoadingListRecordsAction,BLR,2024-05-01T23:00:00.000Z,DEL,2024-05-02T01:50:00.000Z,ABCAD,7MA,Y189
37506,9d1a19e10783012d,received,33845798,2024-04-30 22:14:13,AB,2359,1,BLR,service-acco,AssignLCCAction,BLR,2024-05-01T23:00:00.000Z,DEL,2024-05-02T01:50:00.000Z,ABCAD,7MA,Y189
37505,9d1a19e10783012d,received,33845790,2024-04-30 22:14:13,AB,2359,1,BLR,service-acco,AssignLCCAction,BLR,2024-05-01T23:00:00.000Z,DEL,2024-05-02T01:50:00.000Z,ABCAD,7MA,Y189
480044,9d1a19e10783012d,received,33845791,2024-04-30 22:14:13,AB,2359,1,BLR,service-acco,StoreRegistrationAndConfigurationAc,BLR,2024-05-01T23:00:00.000Z,DEL,2024-05-02T01:50:00.000Z,ABCAD,7MA,Y189
25579,9d1a19e10783012d,saved,33846025,2024-04-30 22:14:13,AB,2359,1,BLR,service-acco,AssignLCCAction,BLR,2024-05-01T23:00:00.000Z,DEL,2024-05-02T01:50:00.000Z,ABCAD,7MA,Y189
480045,9d1a19e10783012d,saved,33846024,2024-04-30 22:14:13,AB,2359,1,BLR,service-acco,StoreRegistrationAndConfigurationAc,BLR,2024-05-01T23:00:00.000Z,DEL,2024-05-02T01:50:00.000Z,ABCAD,7MA,Y189
25577,9d1a19e10783012d,saved,33845799,2024-04-30 22:14:13,AB,2359,1,BLR,service-acco,AssignLCCAction,BLR,2024-05-01T23:00:00.000Z,DEL,2024-05-02T01:50:00.000Z,ABCAD,7MA,Y189


In [13]:
# Zeigt wie oft die einzelnen actions pro flug vorkommen
abcd.drop_duplicates(subset=['flight_id', 'action_name'])['action_name'].value_counts()

action_name
ASMMsgProcessor                        2379
UpdateFlightAction                     1997
AssignLCCAction                        1997
StoreRegistrationAndConfigurationAc    1996
UpdateCrewDataAction                   1992
CalculateWeightAndTrimAction           1992
ResetLoadingListRecordsAction          1489
UpdateLoadTableAction                  1489
TransferCargoAction                     964
ChangeFlightLegStateAction              944
StorePaxDataAction                      741
FuelDataInitializer                      71
CloseLegAction                            1
SendLoadsheetAction                       1
StorePaxDataGuiAction                     1
StoreCKIAverageWeightAction               1
CheckinMsgProcessor                       1
CargoFinalAction                          1
AssignUnassignViewAction                  1
SendLoadingInstructionAction              1
UpdateHistoricalFlightLegInfoAction       1
StoreAircraftDataAction                   1
UpdateFuelDataAction

### >> Finding 2: Wenn man abcd["departureAirport"] + abcd["arrivalAirport"] + abcd["departureTime"] + abcd["arrivalTime"] als index nimmt -> So ergeben sich für den ersten Datensatz (ABCD) 2379 Flüge. 6 Aktionen kommen in >1900 Fällen vor. 